In [6]:
from functools import partial
from modules import dataloaders, schedulers, model, train
from torch import nn, optim
from torchvision import transforms
import torch
import pandas as pd

In [2]:
print(torch.cuda.is_available(), torch.backends.cudnn.enabled)
cuda_flag = torch.cuda.is_available()
device = torch.device('cuda') if cuda_flag else torch.device('cpu')

True True


In [3]:
dl, _ = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10', batch_size=100)
rgb_ave = dataloaders.channel_avg(dl)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
def record_run(train_dl:torch.utils.data.DataLoader,
               test_dl:torch.utils.data.DataLoader,
               net:nn.Module,
               crit:nn.Module,
               optimizer:optim.Optimizer,
               scheduler:optim.lr_scheduler._LRScheduler,
               device:torch.device,
               iterations:int,
               result_path:str):
    'TODO:docstring'
    recorder = {
        'iteration' : [],
        'trn_loss' : [],
        'lr' : [],
        'val_loss' : [],
        'val_acc' : []}
    obe = partial(train.on_batch_end, recorder, test_dl, net, crit, device)
    train.train_run(net, train_dl, crit, opt, scheduler, iterations, obe, device)
    return pd.DataFrame(recorder).to_csv(result_path, index=False)

In [19]:
# fixed
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.004)
sched = schedulers.FixedScheduler(optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           70000,
           './results/fixed.csv')

Files already downloaded and verified
Files already downloaded and verified


2.2959251403808594 | 2.3048656678199766 | 0.1
2.2366421222686768 | 2.252965657711029 | 0.2115
1.9379587173461914 | 1.9531116092205048 | 0.2997
1.8152180910110474 | 1.7316903233528138 | 0.3808
1.6335580348968506 | 1.598079216480255 | 0.4195
1.3986411094665527 | 1.522326169013977 | 0.4467
1.389978289604187 | 1.475188820362091 | 0.4663
1.5397993326187134 | 1.4426518404483795 | 0.4792
1.3852248191833496 | 1.3794143879413605 | 0.5059
1.40769624710083 | 1.3441275775432586 | 0.521
1.2174060344696045 | 1.3135958635807037 | 0.5323
1.3884704113006592 | 1.2910792154073716 | 0.5377
1.1617053747177124 | 1.2626908934116363 | 0.5492
1.2419323921203613 | 1.2411016094684602 | 0.5572
0.9752600789070129 | 1.2126768308877944 | 0.5731
1.1910371780395508 | 1.2149149960279464 | 0.569
1.2233253717422485 | 1.1825607550144195 | 0.5805
1.1781970262527466 | 1.2094527328014373 | 0.5747
1.2020012140274048 | 1.1470334786176681 | 0.5952
1.2650545835494995 | 1.1130068135261535 | 0.6075
1.0533565282821655 | 1.144056512

In [11]:
# triangular 2
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
sched = schedulers.Triangular2Scheduler(step_size=2000, min_lr=0.001, max_lr=0.006, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           25000,
           './results/triangular2.csv')

Files already downloaded and verified
Files already downloaded and verified


2.3136425018310547 | 2.306031334400177 | 0.1
2.238301992416382 | 2.0625839817523954 | 0.269
1.5974258184432983 | 1.6049357092380523 | 0.4127
1.592211127281189 | 1.4361858093738555 | 0.4825
1.1651575565338135 | 1.3220557034015656 | 0.5247
1.2183817625045776 | 1.2084190952777862 | 0.5722
1.2005501985549927 | 1.1189057868719101 | 0.6074
1.274261713027954 | 1.048216148018837 | 0.6323
0.8117294907569885 | 1.0053761899471283 | 0.6509
1.0864410400390625 | 1.011695974469185 | 0.6432
1.0308574438095093 | 0.9823335891962052 | 0.6547
0.8916634917259216 | 0.9875688737630844 | 0.6581
1.0218400955200195 | 1.015904403924942 | 0.6474
1.0009117126464844 | 0.9313847261667252 | 0.6786
1.057079553604126 | 0.8999527567625045 | 0.6876
1.1201342344284058 | 0.8934547179937362 | 0.6942
0.8295238018035889 | 0.8716656118631363 | 0.7008
0.8182371258735657 | 0.8723896157741546 | 0.6976
0.802090048789978 | 0.8740288048982621 | 0.6984
0.9272717833518982 | 0.882567892074585 | 0.6912
0.6563241481781006 | 0.85800519466

In [16]:
# decay
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
sched = schedulers.DecayScheduler(step_size=4000, min_lr=0.001, max_lr=0.007, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           25000,
           './results/decay.csv')

Files already downloaded and verified
Files already downloaded and verified


2.3158750534057617 | 2.3052439451217652 | 0.0939
1.55889892578125 | 1.5285473811626433 | 0.4411
1.4037697315216064 | 1.3739276313781739 | 0.5096
1.0877312421798706 | 1.2430369985103606 | 0.5566
1.0989784002304077 | 1.1535150969028474 | 0.6035
1.1141140460968018 | 1.0960009902715684 | 0.6123
1.0204178094863892 | 1.0451411986351014 | 0.6345
1.0428575277328491 | 1.0014646846055983 | 0.6501
1.1275478601455688 | 0.9752665412425995 | 0.6587
0.8905993700027466 | 0.9625169438123703 | 0.6675
0.9804762005805969 | 0.958123340010643 | 0.664
0.8910219073295593 | 0.9456889796257019 | 0.671
1.0942773818969727 | 0.9407467526197434 | 0.6734
0.8029584288597107 | 0.9291936069726944 | 0.6726
0.8100183606147766 | 0.9213050878047944 | 0.6782
0.9432716965675354 | 0.9157968461513519 | 0.6824
0.9402356147766113 | 0.9094270581007003 | 0.6816
0.7231302857398987 | 0.9107110375165939 | 0.6791
0.7717517018318176 | 0.9013521921634674 | 0.6847
0.9524957537651062 | 0.8880109769105912 | 0.6909
0.8856656551361084 | 0.89

In [17]:
# exp
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
sched = schedulers.ExpScheduler(gamma= 0.99994, initial_lr=0.001, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           70000,
           './results/exp.csv')

Files already downloaded and verified
Files already downloaded and verified


2.302657127380371 | 2.3043775153160095 | 0.1
2.112977981567383 | 2.1080586647987367 | 0.2361
2.0291647911071777 | 1.9572598206996918 | 0.297
1.8521347045898438 | 1.778107168674469 | 0.3617
1.6906859874725342 | 1.6202886533737182 | 0.4084
1.493095874786377 | 1.5431618463993073 | 0.4349
1.3083641529083252 | 1.486073534488678 | 0.4639
1.4849833250045776 | 1.4632676494121553 | 0.471
1.4820462465286255 | 1.415978535413742 | 0.4907
1.299808382987976 | 1.3790383815765381 | 0.4994
1.3899801969528198 | 1.344031810760498 | 0.5161
1.3542070388793945 | 1.3280508637428283 | 0.5193
1.1270562410354614 | 1.3004732346534729 | 0.5335
1.2261919975280762 | 1.2965278446674346 | 0.5355
1.290164828300476 | 1.2761991119384766 | 0.5475
1.32509183883667 | 1.2356091415882111 | 0.5594
1.1426339149475098 | 1.216015647649765 | 0.5685
1.0477409362792969 | 1.1932143938541413 | 0.5808
1.0865533351898193 | 1.1803095656633378 | 0.5855
1.3205336332321167 | 1.162539222240448 | 0.5886
1.0994775295257568 | 1.179560067057609

In [18]:
# exp_range
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.004)
sched = schedulers.ExpRangeScheduler(gamma=0.99994, step_size=2000, min_lr=0.001, max_lr=0.006, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           42000,
           './results/exp_range.csv')

Files already downloaded and verified
Files already downloaded and verified


2.2997262477874756 | 2.3050629186630247 | 0.1001
2.031094551086426 | 1.982711285352707 | 0.2929
1.6606171131134033 | 1.570360518693924 | 0.4239
1.5865789651870728 | 1.435083280801773 | 0.4812
1.403416633605957 | 1.350356469154358 | 0.5039
1.2419596910476685 | 1.2152656489610671 | 0.5667
1.3964972496032715 | 1.1471135938167571 | 0.5953
1.047499418258667 | 1.071091570854187 | 0.6194
1.0053232908248901 | 1.0342403775453568 | 0.6391
1.0206305980682373 | 1.0278595334291458 | 0.6378
1.1891802549362183 | 1.0334520000219345 | 0.6373
1.0883843898773193 | 1.0215561753511428 | 0.6415
1.0264456272125244 | 0.9898825258016586 | 0.6491
0.9975164532661438 | 0.97717837870121 | 0.6568
0.7255896925926208 | 0.9169443231821061 | 0.6828
0.7477964758872986 | 0.8921011412143707 | 0.6941
0.7784454822540283 | 0.8713482481241226 | 0.6983
1.1393011808395386 | 0.8831535476446152 | 0.6962
0.8665239810943604 | 0.9073327118158341 | 0.6831
0.8120568990707397 | 0.8903094011545182 | 0.6945
0.8632910847663879 | 0.8883305